In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pymongo
import datetime as dt
from datetime import datetime

## Extract Victorian Biodiversity Atlas (VBA) fauna data

In [2]:
# Read just the column names in fauna data csv
col_names = pd.read_csv("../data/VBA_2015_2020.csv", nrows = 0).columns
col_names

Index(['RECORD_ID', 'SITE_ID', 'SURVEY_ID', 'PROJECT_ID', 'TAXON_ID',
       'SCI_NAME', 'COMM_NAME', 'RECORDTYPE', 'RELIABILTY', 'TOTALCOUNT',
       'STARTDATE', 'START_YEAR', 'START_MTH', 'ENDDATE', 'END_YEAR',
       'END_MTH', 'LOCN_DESC', 'TAXON_TYPE', 'LONG_DD94', 'LAT_DD94'],
      dtype='object')

In [3]:
# Set data types for columns with data types other than strings
dtypes_dict = {
    "TOTALCOUNT": int,
    "START_YEAR": int,
    "START_MTH": int,
    "END_YEAR": int,
    "END_MTH": int,
    "LONG_DD94": float,
    "LAT_DD94": float
}

In [4]:
# Read in vic fauna csv from 2015 to 2020
fauna_data = pd.read_csv(
    "../data/VBA_2015_2020.csv",
    parse_dates = ["STARTDATE", "ENDDATE"],
    dtype = {col: str for col in col_names if col not in dtypes_dict})
fauna_data.head()

,RECORD_ID,SITE_ID,SURVEY_ID,PROJECT_ID,TAXON_ID,SCI_NAME,COMM_NAME,RECORDTYPE,RELIABILTY,TOTALCOUNT,STARTDATE,START_YEAR,START_MTH,ENDDATE,END_YEAR,END_MTH,LOCN_DESC,TAXON_TYPE,LONG_DD94,LAT_DD94
0,8597419,947931,1405903,4377,1557,Paratya australiensis,Common Freshwater Shrimp,Observation,Confirmed,0,2016-12-16,2016,12,NaT,0,0,McCallum Creek-4_7-TR-16-333,"Mussels, decopod crustacea",143.649002,-37.283901
1,9067844,1084677,1776514,5326,10408,Colluricincla harmonica,Grey Shrike-thrush,Observation with supporting evidence,Acceptable,0,2018-08-28,2018,8,2018-09-25,2018,9,345-513-0003 FSQ1,Passerine birds,145.768997,-37.782501
2,8218590,771970,1221401,4366,10991,Turdus merula,Common Blackbird,Seen,Acceptable,0,2015-04-12,2015,4,NaT,0,0,Ocean Acres Bush Park Nature Reserve,Passerine birds,144.287399,-38.315601
3,9047388,1070861,1760792,5326,11242,Wallabia bicolor,Black-tailed Wallaby,Observation with supporting evidence,Acceptable,0,2018-11-14,2018,11,2018-12-14,2018,12,833-518-0004 BUQ1,Mammals,148.848099,-37.601601
4,9359539,1116727,1809370,5543,10525,Cisticola exilis,Golden-headed Cisticola,Seen,Acceptable,1,2015-05-07,2015,5,2015-05-07,2015,5,MANNIBADAR (581481),Passerine birds,143.481903,-37.781799


## Transform VBA fauna data

In [5]:
# Column Renaming
fauna_df = fauna_data.rename(columns={
    "RECORD_ID": "record_id",
    "SITE_ID": "site_id",
    "SURVEY_ID": "survey_id",
    "PROJECT_ID": "project_id",
    "TAXON_ID": "taxon_id",
    "SCI_NAME": "sci_name",
    "COMM_NAME": "comm_name",
    "RECORDTYPE": "recordtype",
    "RELIABILTY": "reliability",
    "TOTALCOUNT": "totalcount",
    "STARTDATE": "start_date",
    "START_YEAR": "start_year",
    "START_MTH": "start_mth",
    "ENDDATE": "end_date",
    "END_YEAR": "end_year",
    "END_MTH": "end_mth",
    "LOCN_DESC": "location_desc",
    "TAXON_TYPE": "taxon_type",
    "LONG_DD94": "long",
    "LAT_DD94": "lat"})

In [6]:
# Test record_id uniqueness
fauna_df.record_id.is_unique

True

In [7]:
print(f"Number of unique record ids: {fauna_df.record_id.nunique()}")

Number of unique record ids: 346829


In [8]:
# Test survey_id uniqueness
fauna_df.survey_id.is_unique

False

In [9]:
print(f"Number of unique survey ids: {fauna_df.survey_id.nunique()}")

Number of unique survey ids: 97240


In [10]:
# Test site_id uniqueness
fauna_df.site_id.is_unique

False

In [11]:
print(f"Number of unique site ids: {fauna_df.site_id.nunique()}")

Number of unique site ids: 55164


In [12]:
# Test project_id uniqueness
fauna_df.project_id.is_unique

False

In [13]:
print(f"Number of unique project ids: {fauna_df.project_id.nunique()}")

Number of unique project ids: 522


In [14]:
# Test taxon_id uniqueness
fauna_df.taxon_id.is_unique

False

In [15]:
print(f"Number of unique taxon ids: {fauna_df.taxon_id.nunique()}")

Number of unique taxon ids: 941


In [16]:
print(f"Number of unique taxon types: {fauna_df.taxon_type.nunique()}")

Number of unique taxon types: 14


In [17]:
# Reorder the columns
fauna_df = fauna_df[["record_id", "survey_id", "site_id", "project_id", "taxon_id", "taxon_type"
                     ,"comm_name", "sci_name", "totalcount", "location_desc", "long", "lat"
                     ,"end_year", "end_mth", "end_date", "start_year", "start_mth", "start_date"
                     ,"recordtype", "reliability"]]
fauna_df.head()

,record_id,survey_id,site_id,project_id,taxon_id,taxon_type,comm_name,sci_name,totalcount,location_desc,long,lat,end_year,end_mth,end_date,start_year,start_mth,start_date,recordtype,reliability
0,8597419,1405903,947931,4377,1557,"Mussels, decopod crustacea",Common Freshwater Shrimp,Paratya australiensis,0,McCallum Creek-4_7-TR-16-333,143.649002,-37.283901,0,0,NaT,2016,12,2016-12-16,Observation,Confirmed
1,9067844,1776514,1084677,5326,10408,Passerine birds,Grey Shrike-thrush,Colluricincla harmonica,0,345-513-0003 FSQ1,145.768997,-37.782501,2018,9,2018-09-25,2018,8,2018-08-28,Observation with supporting evidence,Acceptable
2,8218590,1221401,771970,4366,10991,Passerine birds,Common Blackbird,Turdus merula,0,Ocean Acres Bush Park Nature Reserve,144.287399,-38.315601,0,0,NaT,2015,4,2015-04-12,Seen,Acceptable
3,9047388,1760792,1070861,5326,11242,Mammals,Black-tailed Wallaby,Wallabia bicolor,0,833-518-0004 BUQ1,148.848099,-37.601601,2018,12,2018-12-14,2018,11,2018-11-14,Observation with supporting evidence,Acceptable
4,9359539,1809370,1116727,5543,10525,Passerine birds,Golden-headed Cisticola,Cisticola exilis,1,MANNIBADAR (581481),143.481903,-37.781799,2015,5,2015-05-07,2015,5,2015-05-07,Seen,Acceptable


In [18]:
# Overview of the fauna data
fauna_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346829 entries, 0 to 346828
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   record_id      346829 non-null  object        
 1   survey_id      346829 non-null  object        
 2   site_id        346829 non-null  object        
 3   project_id     346829 non-null  object        
 4   taxon_id       346829 non-null  object        
 5   taxon_type     346829 non-null  object        
 6   comm_name      346829 non-null  object        
 7   sci_name       346829 non-null  object        
 8   totalcount     346829 non-null  int64         
 9   location_desc  346829 non-null  object        
 10  long           346829 non-null  float64       
 11  lat            346829 non-null  float64       
 12  end_year       346829 non-null  int64         
 13  end_mth        346829 non-null  int64         
 14  end_date       52465 non-null   datetime64[ns]
 15  

In [19]:
# We can see that the TOTALCOUNT of some records is 0. Let's have an overview of them
zero_totalcount = fauna_df[fauna_df["totalcount"] == 0]
zero_totalcount.head(10)

,record_id,survey_id,site_id,project_id,taxon_id,taxon_type,comm_name,sci_name,totalcount,location_desc,long,lat,end_year,end_mth,end_date,start_year,start_mth,start_date,recordtype,reliability
0,8597419,1405903,947931,4377,1557,"Mussels, decopod crustacea",Common Freshwater Shrimp,Paratya australiensis,0,McCallum Creek-4_7-TR-16-333,143.649002,-37.283901,0,0,NaT,2016,12,2016-12-16,Observation,Confirmed
1,9067844,1776514,1084677,5326,10408,Passerine birds,Grey Shrike-thrush,Colluricincla harmonica,0,345-513-0003 FSQ1,145.768997,-37.782501,2018,9,2018-09-25,2018,8,2018-08-28,Observation with supporting evidence,Acceptable
2,8218590,1221401,771970,4366,10991,Passerine birds,Common Blackbird,Turdus merula,0,Ocean Acres Bush Park Nature Reserve,144.287399,-38.315601,0,0,NaT,2015,4,2015-04-12,Seen,Acceptable
3,9047388,1760792,1070861,5326,11242,Mammals,Black-tailed Wallaby,Wallabia bicolor,0,833-518-0004 BUQ1,148.848099,-37.601601,2018,12,2018-12-14,2018,11,2018-11-14,Observation with supporting evidence,Acceptable
6,9067841,1776514,1084677,5326,10488,Passerine birds,White-browed Scrubwren,Sericornis frontalis,0,345-513-0003 FSQ1,145.768997,-37.782501,2018,9,2018-09-25,2018,8,2018-08-28,Observation with supporting evidence,Acceptable
9,8897604,1686135,1000965,4335,528552,Mammals,Red Fox,Vulpes vulpes,0,Basalt 10,144.098206,-37.270500,2018,2,2018-02-07,2018,1,2018-01-17,Observation with supporting evidence,Acceptable
10,9345216,1799930,1107289,2936,5140,Fish,Dry waterbody,Misc Dry,0,Pig and Whistle Creek-2_23-TR-18-421,147.853394,-37.532299,0,0,NaT,2018,11,2018-11-16,Observation,Confirmed
12,8432377,1292811,840246,4551,10364,Passerine birds,Willie Wagtail,Rhipidura leucophrys,0,Bulla Hill and School Hill,144.802200,-37.634102,0,0,NaT,2015,5,2015-05-18,Observation,Acceptable
13,8956790,1715391,1027944,4836,11003,Mammals,Short-beaked Echidna,Tachyglossus aculeatus,0,New Holland Mouse camera survey_Site_NHM423,147.533798,-38.084000,2018,3,2018-03-20,2018,3,2018-03-07,Observation with supporting evidence,Acceptable
15,8994856,1741637,1052493,5326,11115,Mammals,Mountain Brush-tailed Possum,Trichosurus cunninghami,0,298-516-0003,145.521393,-37.442902,2018,8,2018-08-23,2018,7,2018-07-27,Observation with supporting evidence,Acceptable


In [20]:
# Percentage of number of rows with totalcount equal to 0 against total number of rows of the dataframe
(zero_totalcount.shape[0]/fauna_df.shape[0])*100

19.936914156544002

## Filter VBA fauna data against scraped data

In [21]:
# Import the webscraped animal data
scraped_df = pd.read_csv("../data/animal_image_to_merge.csv", dtype="str")
scraped_df.head()

,animal_name,image_url,image_alternative,introduction,threat_paragraph
0,Alpine She-oak Skink,https://www.zoo.org.au/media/2050/1023_alpine_...,Alpine She-oak Skink sunning it self on a rock...,Found in only a few locations in Victoria and ...,Major threats\r\nFire is a huge danger to the ...
1,Baw Baw Frog,https://www.zoo.org.au/media/2052/21295_baw_ba...,Baw Baw Frog resting in bright green moss. Loo...,All estimates point to extinction in the wild ...,Major threats\r\nThe loss of the Baw Baw Frog ...
2,Brush-tailed Rock-wallaby,https://www.zoo.org.au/media/2045/21882_brush-...,Brush Tailed Rock Wallabies resting in the grass.,"In Victoria, the Brush-tailed Rock-wallaby now...",Major threats \r\nChanges to habitat and the i...
3,Eastern Barred Bandicoot,https://www.zoo.org.au/media/2053/4376_eastern...,Small Eastern Barred Bandicoot side view forag...,The Eastern Barred Bandicoot is listed as exti...,The plan for recovery\r\nZoos Victoria has par...
4,Giant Burrowing Frog,https://www.zoo.org.au/media/2056/23479_giant_...,Giant Burrowing Frog on wet rocks side view. T...,Although we know that populations of the Giant...,"['Over the next five years, Zoos Victoria will..."


In [22]:
# Extract list of unique animals of interest
species = scraped_df["animal_name"].unique().tolist()

In [23]:
# Filter the fauna data with the species of interest
short_fauna_df = fauna_df[fauna_df["comm_name"].isin(species)]
short_fauna_df.head()

,record_id,survey_id,site_id,project_id,taxon_id,taxon_type,comm_name,sci_name,totalcount,location_desc,long,lat,end_year,end_mth,end_date,start_year,start_mth,start_date,recordtype,reliability
102,8261909,1225877,833395,4236,11141,Mammals,Leadbeater's Possum,Gymnobelideus leadbeateri,0,DSS3B,145.841904,-37.845600,0,0,NaT,2016,4,2016-04-24,Observation,Confirmed
195,8590335,1401349,943395,4095,11141,Mammals,Leadbeater's Possum,Gymnobelideus leadbeateri,1,DELWP Case Reference number 2017-0060,146.288498,-37.922600,0,0,NaT,2017,8,2017-08-20,Seen,Confirmed
391,6942095,1102090,766516,4078,11141,Mammals,Leadbeater's Possum,Gymnobelideus leadbeateri,1,Mon1_camera1,145.934403,-37.818600,2015,9,2015-09-29,2015,9,2015-09-08,Observation,Confirmed
426,8596613,1405576,947609,1,10309,Non-passerine birds,Swift Parrot,Lathamus discolor,5,"Mclaughlans Lane Pipetrack, Plenty",145.108398,-37.676399,2017,9,2017-09-25,2017,9,2017-09-25,Observation,Acceptable
538,6870927,1085373,716545,4078,11141,Mammals,Leadbeater's Possum,Gymnobelideus leadbeateri,1,40.2_camera2,146.115204,-37.784802,2015,5,2015-05-11,2015,4,2015-04-17,Observation,Confirmed


In [24]:
# Overview of the fauna data after filtering
short_fauna_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2712 entries, 102 to 346720
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   record_id      2712 non-null   object        
 1   survey_id      2712 non-null   object        
 2   site_id        2712 non-null   object        
 3   project_id     2712 non-null   object        
 4   taxon_id       2712 non-null   object        
 5   taxon_type     2712 non-null   object        
 6   comm_name      2712 non-null   object        
 7   sci_name       2712 non-null   object        
 8   totalcount     2712 non-null   int64         
 9   location_desc  2712 non-null   object        
 10  long           2712 non-null   float64       
 11  lat            2712 non-null   float64       
 12  end_year       2712 non-null   int64         
 13  end_mth        2712 non-null   int64         
 14  end_date       899 non-null    datetime64[ns]
 15  start_year     27

In [25]:
# Check for any extreme values
print(f"Maximum total count is: {short_fauna_df.totalcount.max()}")
print(f"Minimum total count is: {short_fauna_df.totalcount.min()}")
print(f"Maximum longitude is: {short_fauna_df.long.max()}")
print(f"Minimum longitude is: {short_fauna_df.long.min()}")
print(f"Maximum latitude is: {short_fauna_df.lat.max()}")
print(f"Minimum latitude is: {short_fauna_df.lat.min()}")
print(f"Maximum end year is: {short_fauna_df.end_year.max()}")
print(f"Minimum end year is: {short_fauna_df.end_year.min()}")
print(f"Maximum end month is: {short_fauna_df.end_mth.max()}")
print(f"Minimum end month is: {short_fauna_df.end_mth.min()}")
print(f"Maximum start year is: {short_fauna_df.start_year.max()}")
print(f"Minimum start year is: {short_fauna_df.start_year.min()}")
print(f"Maximum start month is: {short_fauna_df.start_mth.max()}")
print(f"Minimum start month is: {short_fauna_df.start_mth.min()}")
print(f"Maximum start date is: {short_fauna_df.start_date.max()}")
print(f"Minimum start date is: {short_fauna_df.start_date.min()}")

Maximum total count is: 128
Minimum total count is: 0
Maximum longitude is: 149.9367981
Minimum longitude is: 140.9933014
Maximum latitude is: -34.5940018
Minimum latitude is: -39.0321007
Maximum end year is: 2020
Minimum end year is: 0
Maximum end month is: 12
Minimum end month is: 0
Maximum start year is: 2020
Minimum start year is: 2015
Maximum start month is: 12
Minimum start month is: 1
Maximum start date is: 2020-03-30 00:00:00
Minimum start date is: 2015-01-01 00:00:00


As can be seen, there are a number of records with total count of 0. They are records of surveys with no sightings of a targeted specie. Hence, we'll remove them.

As there are a lot of null end_date values, their extracted end years and end months equal to 0. Hence we might use start date in our time series visualisation. The null end dates might indicate that a survey hasn't ended up to our group's data extraction.

In [26]:
print(f"The number of records with totalcount of zero: {short_fauna_df[short_fauna_df.totalcount == 0].shape[0]}")

The number of records with totalcount of zero: 623


In [27]:
# Values in location description column
short_fauna_df["location_desc"].unique()

array(['DSS3B', 'DELWP Case Reference number 2017-0060', 'Mon1_camera1',
       ..., '316A', '316B', '316C'], dtype=object)

In [28]:
# Values in record types column
short_fauna_df["recordtype"].unique()

array(['Observation', 'Seen', 'Observation with supporting evidence',
       'Captured and released', 'Indirect evidence', 'Heard', 'Captured'],
      dtype=object)

In [29]:
# Values in reliability column
short_fauna_df["reliability"].unique()

array(['Confirmed', 'Acceptable'], dtype=object)

Values in the location description, record types and reliability columns do not seem to be informative enough. Hence we'll remove these columns. We'll also remove columns project_id and site_id as they are not required for our project's purpose.

In [30]:
# Filter out the records with total count of 0 and remove end_year, end_date, end_mth, start_year, start_mth columns
final_fauna_df = short_fauna_df[short_fauna_df.totalcount > 0].drop([
    'site_id', 'project_id', 'location_desc', 'end_year', 'end_mth',\
    'end_date', 'start_year', 'start_mth', 'recordtype', 'reliability'], axis = 1)
final_fauna_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2089 entries, 195 to 346720
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   record_id   2089 non-null   object        
 1   survey_id   2089 non-null   object        
 2   taxon_id    2089 non-null   object        
 3   taxon_type  2089 non-null   object        
 4   comm_name   2089 non-null   object        
 5   sci_name    2089 non-null   object        
 6   totalcount  2089 non-null   int64         
 7   long        2089 non-null   float64       
 8   lat         2089 non-null   float64       
 9   start_date  2089 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 179.5+ KB


In [31]:
# Sort the fauna dataframe by comm_name and start_date
final_fauna_df.sort_values(by=["comm_name", "start_date"], inplace=True)
final_fauna_df.head(10)

,record_id,survey_id,taxon_id,taxon_type,comm_name,sci_name,totalcount,long,lat,start_date
65638,8492998,1343293,13042,Amphibians,Giant Burrowing Frog,Heleioporus australiacus,1,147.462906,-37.719601,2016-03-05
65922,8952200,1714254,13042,Amphibians,Giant Burrowing Frog,Heleioporus australiacus,1,147.628799,-37.372799,2018-06-26
72337,8994935,1741679,527397,Mammals,Greater Glider,fam. Pseudocheiridae gen. Petauroides,1,145.643097,-38.017799,2018-10-19
283231,9752298,1908065,10138,Waders,Hooded Plover,Thinornis cucullatus,1,144.420105,-38.285999,2015-01-01
106178,9755099,1908249,10138,Waders,Hooded Plover,Thinornis cucullatus,2,143.379303,-38.763199,2015-01-06
269706,9754985,1908240,10138,Waders,Hooded Plover,Thinornis cucullatus,4,144.420105,-38.285999,2015-01-06
271437,9754784,1908227,10138,Waders,Hooded Plover,Thinornis cucullatus,4,144.420105,-38.285999,2015-01-06
267889,9761062,1908601,10138,Waders,Hooded Plover,Thinornis cucullatus,1,144.420105,-38.285999,2015-01-24
271167,9761901,1908641,10138,Waders,Hooded Plover,Thinornis cucullatus,2,144.420105,-38.285999,2015-01-26
276315,9762247,1908658,10138,Waders,Hooded Plover,Thinornis cucullatus,2,144.420105,-38.285999,2015-01-26


In [32]:
# Add a new column month_year for aggregation purpose
final_fauna_df["year_month"] = final_fauna_df["start_date"].dt.strftime('%Y-%m')

In [33]:
final_fauna_df.head()

,record_id,survey_id,taxon_id,taxon_type,comm_name,sci_name,totalcount,long,lat,start_date,year_month
65638,8492998,1343293,13042,Amphibians,Giant Burrowing Frog,Heleioporus australiacus,1,147.462906,-37.719601,2016-03-05,2016-03
65922,8952200,1714254,13042,Amphibians,Giant Burrowing Frog,Heleioporus australiacus,1,147.628799,-37.372799,2018-06-26,2018-06
72337,8994935,1741679,527397,Mammals,Greater Glider,fam. Pseudocheiridae gen. Petauroides,1,145.643097,-38.017799,2018-10-19,2018-10
283231,9752298,1908065,10138,Waders,Hooded Plover,Thinornis cucullatus,1,144.420105,-38.285999,2015-01-01,2015-01
106178,9755099,1908249,10138,Waders,Hooded Plover,Thinornis cucullatus,2,143.379303,-38.763199,2015-01-06,2015-01


## Filter webscraped animal image data against VBA fauna data 

In [34]:
final_animal_list = final_fauna_df["comm_name"].unique().tolist()
final_animal_list

['Giant Burrowing Frog',
 'Greater Glider',
 'Hooded Plover',
 "Leadbeater's Possum",
 'Mallee Emu-wren',
 'Mountain Pygmy-possum',
 'New Holland Mouse',
 'Regent Honeyeater',
 'Swift Parrot']

In [35]:
# Number of final animals
len(final_animal_list)

9

In [36]:
# Number of taxon ids
taxon_ids = final_fauna_df["taxon_id"].unique().tolist()
len(taxon_ids)

9

The number of animals is equal to the number of taxon ids. For each animal of interest, there is only one corresponding taxon id.

In [37]:
# Filter the webscraped data to have only the above animals
final_scraped_df = scraped_df[scraped_df["animal_name"].isin(final_animal_list)].copy()
final_scraped_df

,animal_name,image_url,image_alternative,introduction,threat_paragraph
4,Giant Burrowing Frog,https://www.zoo.org.au/media/2056/23479_giant_...,Giant Burrowing Frog on wet rocks side view. T...,Although we know that populations of the Giant...,"['Over the next five years, Zoos Victoria will..."
11,Leadbeater's Possum,https://www.zoo.org.au/media/2057/22861_leadbe...,Close up view of the face of a Leadbeater Poss...,"Once thought to be extinct, the Leadbeater's P...",The major threats\r\nThe loss of hollow-bearin...
13,Mallee Emu-wren,https://www.zoo.org.au/media/1961/23483_mallee...,Mallee Emu-wren in long dry grass looking at t...,The Mallee Emu-wren is particularly vulnerable...,"['In fact, it was a series of fires that cause..."
14,Mountain Pygmy-possum,https://www.zoo.org.au/media/2058/16910_mounta...,Mountain Pygmy Possum standing on its hind leg...,Mountain Pygmy-possums were thought to be exti...,"The major threats \r\nClimate change, the loss..."
15,New Holland Mouse,https://www.zoo.org.au/media/1732/new-holland-...,New Holland Mouse getting a health check wrapp...,The New Holland Mouse is classified as extinct...,The major threats\r\nThe New Holland Mouse is ...
19,Regent Honeyeater,https://www.zoo.org.au/media/2055/22249_regent...,Regent Honeyeater bird on a branch looking dow...,The Regent Honeyeater has been in decline sinc...,The major threats\r\nThe loss of the Box-Ironb...
25,Swift Parrot,https://www.zoo.org.au/media/1960/23484_swift_...,Green Swift Parrot perched on a branch looking...,The Swift Parrot is critically endangered.,['Unless we can solve the issues surrounding i...
27,Greater Glider,https://www.environment.vic.gov.au/__data/asse...,Greater Glider,NaN,Species InformationThe Greater Glider is Austr...
28,Hooded Plover,https://www.environment.vic.gov.au/__data/asse...,Hooded Plover,Species Information\r\n\r\nHooded Plovers are ...,Threats\r\n\r\nHooded Plovers often share thei...


In [38]:
# Overview of the scraped data
final_scraped_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 4 to 28
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   animal_name        9 non-null      object
 1   image_url          9 non-null      object
 2   image_alternative  9 non-null      object
 3   introduction       8 non-null      object
 4   threat_paragraph   9 non-null      object
dtypes: object(5)
memory usage: 432.0+ bytes


In [39]:
# Fill the NaN values with None values for the json-converted file to work
final_scraped_df = final_scraped_df.where(final_scraped_df.notnull(), None)
final_scraped_df

,animal_name,image_url,image_alternative,introduction,threat_paragraph
4,Giant Burrowing Frog,https://www.zoo.org.au/media/2056/23479_giant_...,Giant Burrowing Frog on wet rocks side view. T...,Although we know that populations of the Giant...,"['Over the next five years, Zoos Victoria will..."
11,Leadbeater's Possum,https://www.zoo.org.au/media/2057/22861_leadbe...,Close up view of the face of a Leadbeater Poss...,"Once thought to be extinct, the Leadbeater's P...",The major threats\r\nThe loss of hollow-bearin...
13,Mallee Emu-wren,https://www.zoo.org.au/media/1961/23483_mallee...,Mallee Emu-wren in long dry grass looking at t...,The Mallee Emu-wren is particularly vulnerable...,"['In fact, it was a series of fires that cause..."
14,Mountain Pygmy-possum,https://www.zoo.org.au/media/2058/16910_mounta...,Mountain Pygmy Possum standing on its hind leg...,Mountain Pygmy-possums were thought to be exti...,"The major threats \r\nClimate change, the loss..."
15,New Holland Mouse,https://www.zoo.org.au/media/1732/new-holland-...,New Holland Mouse getting a health check wrapp...,The New Holland Mouse is classified as extinct...,The major threats\r\nThe New Holland Mouse is ...
19,Regent Honeyeater,https://www.zoo.org.au/media/2055/22249_regent...,Regent Honeyeater bird on a branch looking dow...,The Regent Honeyeater has been in decline sinc...,The major threats\r\nThe loss of the Box-Ironb...
25,Swift Parrot,https://www.zoo.org.au/media/1960/23484_swift_...,Green Swift Parrot perched on a branch looking...,The Swift Parrot is critically endangered.,['Unless we can solve the issues surrounding i...
27,Greater Glider,https://www.environment.vic.gov.au/__data/asse...,Greater Glider,None,Species InformationThe Greater Glider is Austr...
28,Hooded Plover,https://www.environment.vic.gov.au/__data/asse...,Hooded Plover,Species Information\r\n\r\nHooded Plovers are ...,Threats\r\n\r\nHooded Plovers often share thei...


## Load

In [40]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collections
db = client.animal_visual_db
vba_fauna = db.vba_fauna
scraped_fauna = db.scraped_fauna

In [41]:
# Drops collections if available to remove duplicates
vba_fauna.drop()
scraped_fauna.drop()

In [42]:
# Load vba fauna data into the vba_fauna collection
vba_fauna.insert_many(final_fauna_df.to_dict('records'))

In [43]:
# Load scraped fauna image and info into the scraped_fauna collection
scraped_fauna.insert_many(final_scraped_df.to_dict('records'))

In [44]:
from pprint import pprint

for record in vba_fauna.find():
    pprint(record)

{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac296'),
 'comm_name': 'Giant Burrowing Frog',
 'lat': -37.7196007,
 'long': 147.4629059,
 'record_id': '8492998',
 'sci_name': 'Heleioporus australiacus',
 'start_date': datetime.datetime(2016, 3, 5, 0, 0),
 'survey_id': '1343293',
 'taxon_id': '13042',
 'taxon_type': 'Amphibians',
 'totalcount': 1,
 'year_month': '2016-03'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac297'),
 'comm_name': 'Giant Burrowing Frog',
 'lat': -37.3727989,
 'long': 147.6287994,
 'record_id': '8952200',
 'sci_name': 'Heleioporus australiacus',
 'start_date': datetime.datetime(2018, 6, 26, 0, 0),
 'survey_id': '1714254',
 'taxon_id': '13042',
 'taxon_type': 'Amphibians',
 'totalcount': 1,
 'year_month': '2018-06'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac298'),
 'comm_name': 'Greater Glider',
 'lat': -38.0177994,
 'long': 145.64309690000002,
 'record_id': '8994935',
 'sci_name': 'fam. Pseudocheiridae gen. Petauroides',
 'start_date': datetime.datetime(2018, 10, 19, 0, 0),
 'survey

{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac2dd'),
 'comm_name': 'Hooded Plover',
 'lat': -38.3139,
 'long': 144.35839840000003,
 'record_id': '9855098',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2015, 12, 13, 0, 0),
 'survey_id': '1915390',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 2,
 'year_month': '2015-12'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac2de'),
 'comm_name': 'Hooded Plover',
 'lat': -38.6453018,
 'long': 143.07000730000001,
 'record_id': '9856312',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2015, 12, 16, 0, 0),
 'survey_id': '1915457',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 1,
 'year_month': '2015-12'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac2df'),
 'comm_name': 'Hooded Plover',
 'lat': -38.428901700000004,
 'long': 144.18099980000002,
 'record_id': '9856361',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2015, 12, 17, 0, 0),
 'survey_id': '1915463',
 'taxon_i

{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac322'),
 'comm_name': 'Hooded Plover',
 'lat': -38.28599929999999,
 'long': 144.420105,
 'record_id': '9949732',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2016, 10, 5, 0, 0),
 'survey_id': '1923160',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 4,
 'year_month': '2016-10'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac323'),
 'comm_name': 'Hooded Plover',
 'lat': -38.428901700000004,
 'long': 144.18099980000002,
 'record_id': '9951171',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2016, 10, 9, 0, 0),
 'survey_id': '1923283',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 4,
 'year_month': '2016-10'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac324'),
 'comm_name': 'Hooded Plover',
 'lat': -38.429798100000006,
 'long': 144.1813965,
 'record_id': '9951431',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2016, 10, 11, 0, 0),
 'survey_id': '1923311',
 'taxon_

 'lat': -38.4174995,
 'long': 144.8222961,
 'record_id': '10007336',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2017, 1, 25, 0, 0),
 'survey_id': '1936946',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 4,
 'year_month': '2017-01'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac365'),
 'comm_name': 'Hooded Plover',
 'lat': -38.28599929999999,
 'long': 144.420105,
 'record_id': '9981966',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2017, 1, 25, 0, 0),
 'survey_id': '1926262',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 3,
 'year_month': '2017-01'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac366'),
 'comm_name': 'Hooded Plover',
 'lat': -38.281299600000004,
 'long': 144.4326935,
 'record_id': '9981983',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2017, 1, 25, 0, 0),
 'survey_id': '1926270',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 3,
 'year_month': '2017-01'}
{'_id': O

{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac3ac'),
 'comm_name': 'Hooded Plover',
 'lat': -38.364399,
 'long': 142.28590390000002,
 'record_id': '10015819',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2017, 4, 22, 0, 0),
 'survey_id': '1940047',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 11,
 'year_month': '2017-04'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac3ad'),
 'comm_name': 'Hooded Plover',
 'lat': -38.5374985,
 'long': 145.3334045,
 'record_id': '10016050',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2017, 4, 24, 0, 0),
 'survey_id': '1940138',
 'taxon_id': '10138',
 'taxon_type': 'Waders',
 'totalcount': 2,
 'year_month': '2017-04'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac3ae'),
 'comm_name': 'Hooded Plover',
 'lat': -37.5387001,
 'long': 149.81390380000002,
 'record_id': '10016319',
 'sci_name': 'Thinornis cucullatus',
 'start_date': datetime.datetime(2017, 4, 29, 0, 0),
 'survey_id': '1940285',
 'taxon_id': '10138',

...

{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac959'),
 'comm_name': 'Mountain Pygmy-possum',
 'lat': -36.990798999999996,
 'long': 147.15800480000001,
 'record_id': '8617172',
 'sci_name': 'Burramys parvus',
 'start_date': datetime.datetime(2017, 11, 23, 0, 0),
 'survey_id': '1410386',
 'taxon_id': '11156',
 'taxon_type': 'Mammals',
 'totalcount': 41,
 'year_month': '2017-11'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac95a'),
 'comm_name': 'Mountain Pygmy-possum',
 'lat': -36.9836006,
 'long': 147.1445007,
 'record_id': '8617169',
 'sci_name': 'Burramys parvus',
 'start_date': datetime.datetime(2017, 11, 24, 0, 0),
 'survey_id': '1410385',
 'taxon_id': '11156',
 'taxon_type': 'Mammals',
 'totalcount': 17,
 'year_month': '2017-11'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac95b'),
 'comm_name': 'Mountain Pygmy-possum',
 'lat': -36.985401200000005,
 'long': 147.1421967,
 'record_id': '8617165',
 'sci_name': 'Burramys parvus',
 'start_date': datetime.datetime(2017, 11, 26, 0, 0),
 'survey_id': '1410384',

{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac9a1'),
 'comm_name': 'Swift Parrot',
 'lat': -38.044899,
 'long': 144.1750946,
 'record_id': '9799952',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2015, 7, 12, 0, 0),
 'survey_id': '1911532',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 6,
 'year_month': '2015-07'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac9a2'),
 'comm_name': 'Swift Parrot',
 'lat': -37.9625015,
 'long': 144.4104004,
 'record_id': '9809188',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2015, 8, 15, 0, 0),
 'survey_id': '1912347',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 4,
 'year_month': '2015-08'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac9a3'),
 'comm_name': 'Swift Parrot',
 'lat': -37.9625015,
 'long': 144.4104004,
 'record_id': '9808692',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2015, 8, 15, 0, 0),
 'survey_id': '1912325',
 'taxon_id': '10309',
 't

 'survey_id': '1928155',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 5,
 'year_month': '2017-09'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac9e6'),
 'comm_name': 'Swift Parrot',
 'lat': -37.676399200000006,
 'long': 145.10839840000003,
 'record_id': '8596613',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2017, 9, 25, 0, 0),
 'survey_id': '1405576',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 5,
 'year_month': '2017-09'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac9e7'),
 'comm_name': 'Swift Parrot',
 'lat': -38.2588997,
 'long': 144.5307007,
 'record_id': '9989442',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2018, 2, 11, 0, 0),
 'survey_id': '1929691',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 1,
 'year_month': '2018-02'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4ac9e8'),
 'comm_name': 'Swift Parrot',
 'lat': -38.2588005,
 'long': 144.530304,
 'record_id': '998

 'comm_name': 'Swift Parrot',
 'lat': -37.7893982,
 'long': 144.57209780000002,
 'record_id': '8943027',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2018, 6, 12, 0, 0),
 'survey_id': '1709982',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 7,
 'year_month': '2018-06'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4aca30'),
 'comm_name': 'Swift Parrot',
 'lat': -36.3793983,
 'long': 145.3592072,
 'record_id': '10057095',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2018, 6, 24, 0, 0),
 'survey_id': '1956303',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 4,
 'year_month': '2018-06'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4aca31'),
 'comm_name': 'Swift Parrot',
 'lat': -36.3793983,
 'long': 145.3592072,
 'record_id': '10057700',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2018, 7, 1, 0, 0),
 'survey_id': '1956528',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 '

 'lat': -38.00719829999999,
 'long': 144.52949519999999,
 'record_id': '10001022',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2019, 4, 20, 0, 0),
 'survey_id': '1934173',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 30,
 'year_month': '2019-04'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4aca6c'),
 'comm_name': 'Swift Parrot',
 'lat': -37.9990005,
 'long': 144.5312958,
 'record_id': '10093681',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2019, 4, 20, 0, 0),
 'survey_id': '1970817',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 20,
 'year_month': '2019-04'}
{'_id': ObjectId('5f6c17a1a31fd3a5ac4aca6d'),
 'comm_name': 'Swift Parrot',
 'lat': -37.9978981,
 'long': 144.5917053,
 'record_id': '10093757',
 'sci_name': 'Lathamus discolor',
 'start_date': datetime.datetime(2019, 4, 20, 0, 0),
 'survey_id': '1970844',
 'taxon_id': '10309',
 'taxon_type': 'Non-passerine birds',
 'totalcount': 20,
 '

In [45]:
for record in scraped_fauna.find():
    pprint(record)

{'_id': ObjectId('5f6c17a1a31fd3a5ac4acabf'),
 'animal_name': 'Giant Burrowing Frog',
 'image_alternative': 'Giant Burrowing Frog on wet rocks side view. The frog '
                      'is dark brown with yellow lips and spots on its side.',
 'image_url': 'https://www.zoo.org.au/media/2056/23479_giant_burrowing_frog_-_credit_required_offsite1.jpg?anchor=center&mode=crop&quality=75&width=2000&height=570&rnd=132131643480000000',
 'introduction': 'Although we know that populations of the Giant Burrowing '
                 'Frog are in decline, this elusive digger is one of '
                 'Victoria’s most poorly understood species.',
 'threat_paragraph': "['Over the next five years, Zoos Victoria will carry out "
                     'important on-the-ground surveys to discover more about '
                     'the Giant Burrowing Frog and its remote habitat. In the '
                     'meantime, we are securing the wellbeing of the '
                     "individual frogs in our

In [46]:
type(vba_fauna.find())

pymongo.cursor.Cursor

In [47]:
type(scraped_fauna.find())

pymongo.cursor.Cursor

## Test aggregations by animal names

In [48]:
# Aggregate total sightings by each animal (represented in common names, science names, taxon ids and taxon types) over 5 years
metadata = list(
    vba_fauna.aggregate(
    [
          {
                  "$group" : {
                      "_id" :"$comm_name",
                      "scientific_name": { "$first": "$sci_name" },
                      "taxon_id": { "$first": "$taxon_id" },
                      "taxon_type": { "$first": "$taxon_type" },
                      "total_sightings": { "$sum": "$totalcount" }
                  }
          }
    ]))

metadata

[{'_id': 'Mountain Pygmy-possum',
  'scientific_name': 'Burramys parvus',
  'taxon_id': '11156',
  'taxon_type': 'Mammals',
  'total_sightings': 823},
 {'_id': 'Greater Glider',
  'scientific_name': 'fam. Pseudocheiridae gen. Petauroides',
  'taxon_id': '527397',
  'taxon_type': 'Mammals',
  'total_sightings': 1},
 {'_id': 'Regent Honeyeater',
  'scientific_name': 'Anthochaera phrygia',
  'taxon_id': '10603',
  'taxon_type': 'Passerine birds',
  'total_sightings': 41},
 {'_id': 'Giant Burrowing Frog',
  'scientific_name': 'Heleioporus australiacus',
  'taxon_id': '13042',
  'taxon_type': 'Amphibians',
  'total_sightings': 2},
 {'_id': 'Hooded Plover',
  'scientific_name': 'Thinornis cucullatus',
  'taxon_id': '10138',
  'taxon_type': 'Waders',
  'total_sightings': 3208},
 {'_id': 'Mallee Emu-wren',
  'scientific_name': 'Stipiturus mallee',
  'taxon_id': '10527',
  'taxon_type': 'Passerine birds',
  'total_sightings': 722},
 {'_id': 'New Holland Mouse',
  'scientific_name': 'Pseudomys n

In [49]:
# Aggregate records by animal name
records_by_animal = list(vba_fauna.aggregate([
    {
        "$group" : {
            "_id" : "$comm_name",
            "record_id": { "$push": "$record_id" },
            "survey_id": { "$push": "$survey_id" },
            "number_sightings": { "$push": "$totalcount" },
            "long": { "$push": "$long" },
            "lat": { "$push": "$lat" },
            "start_date": { "$push": "$start_date" }
        }
    }
]))

records_by_animal[2]

{'_id': 'Regent Honeyeater',
 'record_id': ['10014718',
  '10015751',
  '10016087',
  '10016095',
  '10017362',
  '10017675',
  '10018806',
  '10018932',
  '10029738',
  '10031967',
  '10036445',
  '10054989'],
 'survey_id': ['1939609',
  '1940026',
  '1940157',
  '1940162',
  '1940695',
  '1940800',
  '1941200',
  '1941255',
  '1945556',
  '1946456',
  '1948156',
  '1955342'],
 'number_sightings': [1, 2, 2, 2, 9, 2, 8, 7, 2, 2, 2, 2],
 'long': [146.6318054,
  146.6318054,
  146.6318054,
  146.63279719999997,
  146.6318054,
  146.6553955,
  146.6622925,
  146.6318054,
  146.6737976,
  146.32060239999998,
  148.1578064,
  146.661499],
 'lat': [-36.1671982,
  -36.1671982,
  -36.1671982,
  -36.1651001,
  -36.1671982,
  -36.141201,
  -36.1400986,
  -36.1671982,
  -36.1417999,
  -36.3634987,
  -37.7052002,
  -36.1310005],
 'start_date': [datetime.datetime(2017, 4, 12, 0, 0),
  datetime.datetime(2017, 4, 22, 0, 0),
  datetime.datetime(2017, 4, 24, 0, 0),
  datetime.datetime(2017, 4, 24, 0, 0

In [50]:
# Aggregrate total sightings for each animal by month
sightings_by_month = list(vba_fauna.aggregate([
    {
        "$group": {
            "_id": {
                "animal_name": "$comm_name",
                "year_month": "$year_month"
            },
            "total_sightings": { "$sum": "$totalcount" }
        }
    }
]))
sightings_by_month[0:10]

[{'_id': {'animal_name': 'Hooded Plover', 'year_month': '2015-11'},
  'total_sightings': 23},
 {'_id': {'animal_name': 'Mallee Emu-wren', 'year_month': '2017-02'},
  'total_sightings': 5},
 {'_id': {'animal_name': 'New Holland Mouse', 'year_month': '2015-03'},
  'total_sightings': 33},
 {'_id': {'animal_name': 'Hooded Plover', 'year_month': '2015-07'},
  'total_sightings': 10},
 {'_id': {'animal_name': "Leadbeater's Possum", 'year_month': '2018-10'},
  'total_sightings': 15},
 {'_id': {'animal_name': "Leadbeater's Possum", 'year_month': '2019-10'},
  'total_sightings': 12},
 {'_id': {'animal_name': 'Regent Honeyeater', 'year_month': '2017-06'},
  'total_sightings': 15},
 {'_id': {'animal_name': 'Mallee Emu-wren', 'year_month': '2018-02'},
  'total_sightings': 3},
 {'_id': {'animal_name': 'Swift Parrot', 'year_month': '2016-09'},
  'total_sightings': 3},
 {'_id': {'animal_name': 'Swift Parrot', 'year_month': '2017-04'},
  'total_sightings': 29}]

In [51]:
# Remove "_id" field in vba_fauna to make the table more presentable
data_table = list(vba_fauna.aggregate([ { "$unset": ["_id"] } ]))
data_table[0:2]

[{'record_id': '8492998',
  'survey_id': '1343293',
  'taxon_id': '13042',
  'taxon_type': 'Amphibians',
  'comm_name': 'Giant Burrowing Frog',
  'sci_name': 'Heleioporus australiacus',
  'totalcount': 1,
  'long': 147.4629059,
  'lat': -37.7196007,
  'start_date': datetime.datetime(2016, 3, 5, 0, 0),
  'year_month': '2016-03'},
 {'record_id': '8952200',
  'survey_id': '1714254',
  'taxon_id': '13042',
  'taxon_type': 'Amphibians',
  'comm_name': 'Giant Burrowing Frog',
  'sci_name': 'Heleioporus australiacus',
  'totalcount': 1,
  'long': 147.6287994,
  'lat': -37.3727989,
  'start_date': datetime.datetime(2018, 6, 26, 0, 0),
  'year_month': '2018-06'}]